Made with Tommy Ransbotyn

In [ ]:
import time
#clear every hundred iterations
#import os
#this does not seem to work so supressing prints
# Set up (gonna copy from manual.ipynb)
from optibook.synchronous_client import Exchange

import logging
logger = logging.getLogger('client')
logger.setLevel('ERROR')

print("Setup was successful.")

e = Exchange()
a = e.connect()

#the global max limit for a position
MAXLIMIT = 190
#kindof sigmaesque
def sigmoid(L):
    return(3*(1-abs(L)/4)*L/(1+abs(2*L)))
S1 = sigmoid(1)
def limtransform(lmt):
    L = lmt/(2*MAXLIMIT)
    return((2*MAXLIMIT)*(S1-sigmoid(1-L)))
#none larger than 1
def margintransform(m):
    return(min((sigmoid(m)/S1),1))
# Actually do the arbitrage trades, buying from A and selling to B
#returns volAbuy, volAsell, volBbuy, volBsell to execute
def functradevols(askA, askAvol, bidA, bidAvol, posA, askB, askBvol, bidB, bidBvol, posB, ia, ib):
    
    m = max((bidB - askA), (bidA - askB))
    #print('max margin is, ', round1(m))
    #print('bid B', round1(bidB), 'ask A', round1(askA))
    #print('bid A', round1(bidA), 'ask B', round1(askB))
    #no margin means make no trades
    if m <= 0: 
        return(0,0,0,0,0)
    #sell B buy A
    if bidB > askA: 
       # print('sell B, buy A')
        lmt = min((MAXLIMIT+posB),(MAXLIMIT-posA))
        L = (limtransform(lmt)*margintransform(m))//1
        vol = min(bidBvol, askAvol, L, lmt)//1
        return(ia, ib, askA, bidB, vol)

    #sell A buy B
    else:
       # print('sell A, buy B')
        lmt = min((MAXLIMIT+posA),(MAXLIMIT-posB))
        L = (limtransform(lmt)*margintransform(m))//1
        vol = min(bidAvol, askBvol, L, lmt)//1
        return(ib, ia, askB, bidA, vol)


def totalPositions(instrument_id):
    pos = e.get_positions()[instrument_id]
    
    orders = e.get_outstanding_orders(instrument_id)
    
    for o in orders.values():
        if (o.side == 'bid'):
            pos += o.volume
        else:
            pos -= o.volume
    
    return pos

class Arbitrage:
    def __init__(self, instrumentA, instrumentB):
        self.instrumentA = instrumentA
        self.instrumentB = instrumentB
        
        self.previousBuyId = None
        self.previousSellId = None
    
    # Checks whether we can do an arbitrage trade from buying from 1 and selling to 2
    def checkArbitrage(self, ia, ib):
        
        bookA = e.get_last_price_book(ia)
        asksA = bookA.asks
        askA = asksA[0].price
        askAvol = asksA[0].volume
        bidsA = bookA.bids
        bidA = bidsA[0].price
        bidAvol = bidsA[0].volume
        posA = totalPositions(ia)
        
        bookB = e.get_last_price_book(ib)
        asksB = bookB.asks
        askB = asksB[0].price
        askBvol = asksB[0].volume
        bidsB = bookB.bids
        bidB = bidsB[0].price
        bidBvol = bidsB[0].volume
        posB = totalPositions(ib)
        

        #if (len(asksA) == 0 or len(bidsB) == 0 or len(bidsA) == 0 or len(asksB) == 0):
        #    return
        
        print('\n')
        print('A', posA, 'B', posB)
        
        if (posA+posB)>= 1:
           # print('Correcting Delta sell', posA, posB)
            if (bidA>bidB):
                self.DeltaSellA = e.insert_order(self.instrumentA, price=bidA, volume=(posA+posB), side='ask', order_type='ioc')

            else:
                self.DeltaSellB = e.insert_order(self.instrumentB, price=bidB, volume=(posA+posB), side='ask', order_type='ioc')
        if (posA+posB)<-7:  
          #  print('Correcting Delta buy', posA, posB)
            if (askA<askB):
                self.DeltaBuyA = e.insert_order(self.instrumentA, price=askA, volume=-(posA+posB), side='bid', order_type='ioc')

            else:
                self.DeltaBuyB = e.insert_order(self.instrumentB, price=askB, volume=-(posA+posB), side='bid', order_type='ioc')
                    

        else:
          #  print('checking for trades')
            buybook, sellbook, buyprice, sellprice, v = functradevols(askA, askAvol, bidA, bidAvol, posA, askB, askBvol, bidB, bidBvol, posB, ia, ib)
          #  print('volume', v)
            if v == 0: 
                return
            
            else: 
                #return
                self.doArbitrage(buybook, sellbook, buyprice, sellprice, v)

        return
    

        
    #consider improving using continue calls
    
    def doArbitrage(self, buybook, sellbook, buyprice, sellprice, v):
        print(f"Buying {v} from {buybook} at {buyprice:.1f}, selling to {sellbook} at {sellprice:.1f}")

        #global arb
        #arb = 1
        self.previousBuyId = e.insert_order(buybook, price=buyprice, volume=v, side='bid', order_type='ioc')
        self.previousSellId = e.insert_order(sellbook, price=sellprice, volume=v, side='ask', order_type='ioc')
        return
    
    def fixAllOutstanding(self, instrument_id):
        orders = e.get_outstanding_orders(instrument_id)
        
        volume = 0
        
        for o in orders.values():
            if (o.side == 'bid'):
                volume += o.volume
            else:
                volume -= o.volume
        
        if (volume != 0):
            if (self.canTradeASAP(volume)):
                e.delete_orders(instrument_id)
                self.tradeASAP(volume)
    
    # Checks if the previous trade is outstanding
    # Return 0 if it's not, otherwise return the volume remaining
    # +ve if we have position, -ve if we don't
    def checkPrevious(self, instrument_id):
        orders = e.get_outstanding_orders(instrument_id)
        
        # The previous buy didn't go through
        noBuy = False
        buyId = None
        buyVolumeRem = 0
        
        # The previous sell didn't go through
        noSell = False
        sellId = None
        sellVolumeRem = 0
        
        previousBuy = self.previousBuyId
        previousSell = self.previousSellId
    
        for o in orders.values():
            # print(f"{o.instrument_id}: {o.volume}")
            if (self.previousBuyId and o.order_id == self.previousBuyId):
                noBuy = True
                buyId = o.instrument_id
                buyVolumeRem = o.volume
                
            elif (self.previousSellId and o.order_id == self.previousSellId):
                noSell = True
                sellId = o.instrument_id
                sellVolumeRem = o.volume
                
            else:
                print(f"ERROR: id: {o.order_id}, buy: {previousBuy}, sell: {previousSell}")
            
        if (buyVolumeRem > sellVolumeRem):
            if (self.tradeASAP(buyVolumeRem - sellVolumeRem)):
                if (previousBuy and buyId):
                    e.delete_order(buyId, order_id=previousBuy)
                if (previousSell and sellId):
                    e.delete_order(sellId, order_id=previousSell)
        
        elif (buyVolumeRem < sellVolumeRem):
            if (self.tradeASAP(buyVolumeRem - sellVolumeRem)):
                if (previousBuy and buyId):
                    e.delete_order(buyId, order_id=previousBuy)
                if (previousSell and sellId):
                    e.delete_order(sellId, order_id=previousSell)
            
        elif (noBuy and noSell):
            if (previousBuy and buyId):
                e.delete_order(buyId, order_id=previousBuy)
            if (previousSell and sellId):
                e.delete_order(sellId, order_id=previousSell)
                
    def canTradeASAP(self, volume):
        bookA = e.get_last_price_book(self.instrumentA)
        bookB = e.get_last_price_book(self.instrumentB)
        
        if (volume > 0):
            # Need to buy for as cheap as possible
            asksA = bookA.asks
            asksB = bookB.asks
            
            if (len(asksA) == 0 and len(asksB) == 0):
                return False
                
        elif (volume < 0):
            # Need to sell for as expensive as possible
            bidsA = bookA.bids
            bidsB = bookB.bids
            
            if (len(bidsA) == 0 and len(bidsB) == 0):
                return False
        
        return True

    # Buy or sell the position immediately for the best possible price
    # +ve volume means buy, -ve means sell
    def tradeASAP(self, volume):
        print(f"Trading {volume} ASAP")
        bookA = e.get_last_price_book(self.instrumentA)
        bookB = e.get_last_price_book(self.instrumentB)
        
        if (volume > 0):
            # Need to buy for as cheap as possible
            asksA = bookA.asks
            asksB = bookB.asks
            
            if (len(asksA) == 0 and len(asksB) == 0):
                return False
            
            if (len(asksB) == 0 or asksA[0].price < asksB[0].price):
                # Either no one is selling B or A is cheaper right now
                self.previousBuyId = e.insert_order(self.instrumentA, price=asksA[0].price, volume=volume, side='bid', order_type='limit')
            else:
                # Someone is selling B and B is cheaper right now
                self.previousBuyId = e.insert_order(self.instrumentB, price=asksB[0].price, volume=volume, side='bid', order_type='limit')
            
        elif (volume < 0):
            # Need to sell for as expensive as possible
            bidsA = bookA.bids
            bidsB = bookB.bids
            
            if (len(bidsA) == 0 and len(bidsB) == 0):
                return False
            
            if (len(bidsB) == 0 or bidsA[0].price > bidsB[0].price):
                # Either no one is selling B or A is more expensive right now
                self.previousSellId = e.insert_order(self.instrumentA, price=bidsA[0].price, volume=-volume, side='ask', order_type='limit')
            else:
                # Someone is selling B and B is more expensive right now
                self.previousSellId = e.insert_order(self.instrumentB, price=bidsB[0].price, volume=-volume, side='ask', order_type='limit')
        
        return True
        
    def doTick(self):
        # self.checkPrevious(self.instrumentA)
        # self.checkPrevious(self.instrumentB)
        
        #self.fixAllOutstanding(self.instrumentA)
        #self.fixAllOutstanding(self.instrumentB)
        #print('\n')
        self.checkArbitrage(self.instrumentA, self.instrumentB)
        #self.checkArbitrage(self.instrumentB, self.instrumentA)
        

arbitrage = Arbitrage("PHILIPS_A", "PHILIPS_B")
arb = 0
#pcount = 0
while True:
    time.sleep(0.2)
    pcount = pcount+1
    #if pcount == 5:
    #    os.system('clear')
     #   pcount = 0
    #arbitrage.doTick()
    try:   
        arbitrage.doTick()
        #if arb == 1:
        #   print(e.get_positions())
        #    arb = 0
    except: continue